# ART Docs 

ART (Agent Reinforcement Trainer) : An open-source framework for LLM reinforcement learning using GRPO


GRPO를 사용하는 LLM 강화학습을 위한 open-source 프레임워크인 ART를 이용하여 

자신만의 multi-turn agents를 훈련시켜봅시다.

ART (Agent Reinforcement Trainer)는 Agentic LLMs을 학습시키기 위한 open-source training framework로

**경험** 을 통해 **성능과 신뢰성**을 향상시킬 수 있습니다.

ART는 GRPO (Group Relative Policy Optimization)과 같은 강화학습 기술들을 편리하게 wrapper로 제공하여 최소한의 훈련 비용으로 모델 성능의 극적인 향상을 이룰 수 있게 합니다. 

# Why ART? 

- ART는 기존에 존재하는 애플리케이션에 RL training을 도입하기 위한 편리한 wrappers들을 제공합니다. 이는 개발자의 코드와 상호 작용할 필요가 없는 모듈식 서비스로 훈련 서버를 추상화합니다.

- **Train from anywhere.** 노트북에서 ART 클라이언트를 실행하고 ART 서버가 임시 GPU 지원 환경을 시작하도록 하거나 로컬 GPU에서 실행하세요.

- W&B, Langfuse, OpenPipe와 같은 호스팅 플랫폼과의 통합은 유연한 관찰성을 제공하고 디버깅을 간소화합니다.

- ART는 지능형 기본값을 통해 사용자 정의가 가능합니다. 특정 요구 사항에 맞게 학습 매개변수와 추론 엔진 구성을 구성하거나, 학습 효율성과 안정성을 위해 최적화된 기본값을 활용할 수 있습니다.

# What is RL and when should I use it?

RL (reinforcement learning)은 AI model 자신의 경험으로부터 학습할 수 있게하는 training 기법 중 하나입니다.

RL을 기존 LLM에 적용하는 것은 다음과 같은 것들을 할 수 있습니다: 
- 전반적인 agent 신뢰성 향상 
- QA 또는 생산 과정에서 발견된 특정 실수를 수정
- 사용자에게 배포하기 전에 agent 성능의 신뢰성 구축 

Examples:
- knowledge store로부터 search하고 정보를 추출하기 위한 deep research agent 훈련 
- 새로운 training examples를 추가함으로써 모델 behavior에 발생하는 짜증나는 버그 해결
- 매번 스크립트를 따르는 초고속 음성 에이전트 만들기


# What do I need in order to use RL?


### 필요한것 
- ✅ 하나 또는 그 이상의 LLM을 사용할 프로젝트
- ✅ LLM이 처리해야 할 시나리오 종류에 대한 지식

### 필요하지 않는 것 
- ❌ training dataset 
- ❌ 완성된 reward (보상) 함수 

# Tutorial: Implementing OpenPipe ART with Weights & Biases

해당 튜토리얼에서 우리는 ART를 simple agent task에 적용하고 Weights & Biases (W&B)를 사용하여 training progress를 추적할 것입니다. 

우리 agent의 task는 다음과 같이 직관적입니다: 간단한 곱셈 문제를 푸는것

처음에는, 우리의 language model은 곱셈을 하는데 실수를 할것입니다, 그러나 ART를 이용하면, 이것을 훈련시키며 개선시킬 것입니다. 우리는 W&B Weave와 W&B Models들로 어떻게 지표를 모니터링하는지 알려줄 것이며, trained model을 다루는 방법을 알려줄 것입니다. 

필수조건: W&B 계정이 있어야하고 Python이 설치되어 있어야 합니다. 당신은 당신의 local machone에서 강력한 GPU를 준비할 필요 없습니다 - 우리는 로컬 설정을 사용하여 시연해 보겠습니다. 하지만 GPU가 없다면 SkyPilot 백엔드를 통해 원격 GPU를 사용하도록 ART를 구성할 수 있습니다.

## Step 1 : Install OpenPipe ART and set up W&B

먼저, OpenPipe ART 패키지를 설치하세요 그리고 실험 추적으로 위해 weights & Biases에 로그인합니다. 
당신은 pip를 통해 openpipe-art를 설치할 수 있습니다. 또한, W&B SDK (wandb)를 설치하세요.

In [1]:
!pip install -q openpipe-art wandb

In [2]:
!pip install -q huggingface_hub transformers

Python script 또는 notebook에서, W&B를 로그인 하고 초기화 실행을 하세요: 

In [3]:
from tqdm import tqdm

In [4]:
import wandb


wandb.login()  # you'll be prompted to enter your W&B API key (from your W&B account page)
wandb.init(project="my-agentic-task", name="openpipe-art-demo")

wandb: Currently logged in as: hyunew to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`wandb.init` 호출은 프로젝트(이 경우 "my-agentic-task")와 추적을 위한 실행 이름을 설정합니다. 이 작업이 완료되면 로깅하는 모든 지표가 W&B 대시보드에 실시간으로 전송됩니다. (W&B를 사용하지 않으려면 로그인/초기화 단계를 건너뛸 수 있습니다. ART는 이 단계 없이도 작동합니다. 하지만 교육 과정에 대한 통찰력을 얻으려면 이 단계를 적극 권장합니다.)

## Step 2 : Initialize the ART model and backendNext

우리는 trainable model과 training backend를 설정합니다. 

파인튜닝 하기위해 base model을 지정해주세요. OpenPipe ART는 많은 Hugging Face-compatible LLMs들을 지원합니다; 여기서 우리는 데모를 위해, (상대적으로 lightweight model인) Qwen 3B instruct model과 같은 smaller model을 사용합니다. 우리는 또한 model 이름을 지어주고 로그인을 위해 W&B project를 지정해줍니다.


백엔드의 경우, 로컬 백엔드(사용 가능한 GPU가 있다는 가정 하에)로 시작하겠습니다. 로컬에 GPU가 없다면 SkyPilotBackend를 사용하여 원격 백엔드를 실행할 수 있습니다.

In [5]:
import art  # import the OpenPipe ART library
from art.local import LocalBackend

model_name = "agent-001"
project_name = "my-agentic-task"
base_model_name = "Qwen/Qwen2.5-3B-Instruct"


# Create a trainable model instance
model = art.TrainableModel(
    name=model_name,                # an arbitrary name for your model (used for logging)
    project=project_name,       # W&B project name for grouping runs
    base_model=base_model_name,    # base model to fine-tune (needs to be a model identifier or path)
)


# Set up the training backend
backend = LocalBackend()  # use local GPU; ensure your machine has a GPU and drivers if using this


# If no local GPU, you can use SkyPilotBackend as follows:
# from art.skypilot import SkyPilotBackend
# backend = await SkyPilotBackend.initialize_cluster(cluster_name="art-demo", gpu="A10")  
# (The above would asynchronously provision a remote GPU machine, e.g., with an Nvidia A10 card.)


# Register the model with the backend (prepare the backend server)
await model.register(backend)


/home/khw/miniconda3/envs/openpipe/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 09-19 17:55:42 [__init__.py:235] Automatically detected platform cuda.


/home/khw/miniconda3/envs/openpipe/lib/python3.11/site-packages/art/__init__.py:10: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth  # type: ignore # noqa: F401


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 09-19 17:55:50 [__init__.py:235] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.8.6: Fast Qwen2 patching. Transformers: 4.53.2. vLLM: 0.10.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 2. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 78.07%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.39 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 15.50it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.04it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.04it/s]



INFO 09-19 17:56:21 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 09-19 17:56:22 [model_runner.py:1115] Model loading took 2.2562 GiB and 3.414136 seconds
INFO 09-19 17:56:26 [worker.py:295] Memory profiling takes 3.73 seconds
INFO 09-19 17:56:26 [worker.py:295] the current vLLM instance can use total_gpu_memory (39.39GiB) x gpu_memory_utilization (0.78) = 30.75GiB
INFO 09-19 17:56:26 [worker.py:295] model weights take 2.26GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 2.69GiB; the rest of the memory reserved for KV Cache is 25.71GiB.
INFO 09-19 17:56:26 [executor_base.py:113] # cuda blocks: 46811, # CPU blocks: 10922
INFO 09-19 17:56:26 [executor_base.py:118] Maximum concurrency for 32768 tokens per request: 22.86x
INFO 09-19 17:56:31 [vllm_utils.py:671] Unsloth: Running patched vLLM v0 `capture_model`.
INFO 09-19 17:56:31 [model_runner.py:1385] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To 

Capturing CUDA graph shapes: 100%|██████████| 43/43 [00:16<00:00,  2.56it/s]


INFO 09-19 17:56:48 [model_runner.py:1537] Graph capturing finished in 17 secs, took 5.75 GiB
INFO 09-19 17:56:48 [vllm_utils.py:678] Unsloth: Patched vLLM v0 graph capture finished in 17 secs.
INFO 09-19 17:56:49 [llm_engine.py:424] init engine (profile, create kv cache, warmup model) took 27.12 seconds
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm', 'q_norm', 'k_norm']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm', 'q_norm', 'k_norm']


Unsloth 2025.8.6 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


- `TrainableModel` 는 model을 셋업합니다. 우리는 `base_model="Qwen/Qwen2.5-3B` 를 예제로 제공합니다. 이는 ART 시스템에서 접근 가능한 모델 체크포인트와 일치해야 합니다 (Hugging Face Hub의 공개 모델인 경우 ART에서 다운로드 할 수 있습니다). 또한 당신이 fine-tuning하기 원하는 다른 모델로 변경할 수 있습니다. 

- training을 위해 local server process를 런칭할 `LocalBackend` 를 만듭니다. ` await model.register(backend)` 를 호출하면, ART는 백앤드를 시작합니다. (백앤드의 vLLM 서버에서 모델을 로딩하고 훈련할 준비를 합니다). 이 step은 model weights를 메모리에 로드하기 때문에, 초기에 시간을 조금 차지합니다.

- Note: `await` 는 `register`가 asynchronous operation 임을 나타냅니다.  top-level `await`를 지원하는 노트북 환경에서는 문제가 없습니다. 일반적인 Python script에서는 `asyncio` event loop 내에서 이를 실행해야 합니다. 간단하게, 이 코드는 notebook 안에 또는 async context 안에 있다고 가정합니다. (만약, script를 사용하면, 이러한 호출을 `async def main()`으로 wrapping하고 `asyncio.run(main())`을 사용합니다.)

이제 모델은 사용할 준비가 되었습니다. 추론과 학습을 처리할 수 있는 백엔드에 연결되었습니다. 이제 강화 학습 루프를 시작할 수 있습니다.

## Step 3: Define the task and the reward function

해당 튜토리얼에서 우리의 task는 간단합니다: 2개의 수가 주어지면, agent (model)은 이 두 수의 곱셈을 출력합니다. 우리는 해당 모델에 대해 question-answer interaction으로 구조화합니다.

예를들어, 우리는 "What is 12*13?" 을 제시하면, 대답으로 "156" 을 예측합니다. 만약 모델의 대답이 옳다면, 우리는 positive reward를 주고, 만일 틀렸다면, 우리는 negative reward를 줄 것입니다.

시간이 지남에 따라, 모델은 더 많은 보상을 받기 위해 출력을 조정해야 합니다(즉, 보이는 숫자 범위에 맞게 올바르게 곱하는 법을 배워야 합니다).

우리는 rollout function을 정의해야합니다: 이 function은 agent의 한 episode/interaction을 실행하고 `Trajectory` (메시지 시퀀스 그리고 final reward) 를 return 합니다. 
ART에서, 일반적으로 `model.openai_clinet()`를 사용하여 모델과 interact할 수 있는 **rollout function**을 작성합니다.

아래는 곱셈 작업에 대한 단순화된 rollout function입니다.

In [6]:
import random
import math


# Define one scenario rollout for the multiplication task
async def rollout(model: art.TrainableModel) -> art.Trajectory:
    # Step 1: Prepare a random multiplication problem
    a = random.randint(1, 20)
    b = random.randint(1, 20)
    question = f"What is {a} * {b}?"
    
    # Step 2: Get the model's answer
    openai_client = model.openai_client()
    messages = [{"role": "user", "content": question}]
    
    completion = await openai_client.chat.completions.create(
        model=model.name,
        messages=messages,
        max_tokens=10
    )
    answer = completion.choices[0].message.content.strip()

    # Step 3: Calculate reward
    correct_answer = str(a * b)
    reward_value = 1.0 if answer == correct_answer else -1.0

    # Step 4: 올바른 Trajectory 생성
    trajectory = art.Trajectory(
        messages_and_choices=[  # messages 대신 messages_and_choices 사용
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer}
        ],
        reward=reward_value
    )
    
    return trajectory

위 rollout 코드를 파해쳐봅시다.

- 곱셈 문제를 풀기 위해 `a`와 `b` 2개의 숫자를 랜덤으로 생성합니다.
- 이 숫자들의 곱을 묻는 user message를 구성합니다. (더 복잡한 agent에서는 instruction을 포함한 system message도 추가할 수 있지만, 여기서는 아주 간단하게 구성합니다)
- 우리의 모델에서 `openai_client`를 가져오고, `chat.completions.create` 메서드를 사용하여 모델의 응답을 가져옵니다. `model=model.name`을 지정하여 요청이 백엔드에 등록한 fine-tuning 인스턴스로 전달되도록 합니다.
- 모델의 대답이 캡쳐됩니다. formatting 문제를 방지하기 위해 해당 답변을 제거합니다.
- 그런다음 reward를 결정합니다: 만일 대답이 정답이면, reward = +1, 아니면, -1로 매핑합니다. (우리는 간단한 reward 스키마를 사용합니다; 더 복잡한 task는 더 복잡한 scoring이 필요합니다.)
- 우리는 `art.Trajectory` object를 생성하여 interaction을 기록합니다. 우리는 user question과 assistant's answer를 모두 messages에 포함합니다. 그런다음, 우리는 trajectory에 reward를 할당합니다.
- trajectory가 return되어, 해당 환경에서 agent의 한 rollout이 캡슐화됩니다.

## Step 4: Run training iterations

정의된 rollout function과 함꼐, 여러 iterations를 통해 모델을 훈련시킬 수 있습니다.
강화 학습의 관점에서 보면, 우리는 일련의 에피소드를 진행하고, 각 에피소드 배치가 끝날 때마다 모델을 업데이트합니다.

데모를 위해, 우리는 50번의 training setip을 실행합니다. 각 스텝에서, 우리는 여러 곱셈 문제들을 agent에게 제공합니다 (update 전에 충분한 경험을 수집하기 위해)


OpenPipe ART를 사용하여, model을 training 하는 것은 


OpenPipe ART를 사용하면 롤아웃에서 train method를 호출하는 것만큼 쉽게 모델을 training 할 수 있습니다.
예를 들어, ART를 사용하면 여러 롤아웃을 그룹화하고 내부적으로 training을 처리할 수 있습니다.

아래는 training에 대한 pseudo code 입니다.

In [7]:
import asyncio

In [8]:
# Train the model for a certain number of steps
training_steps = 50
rollouts_per_step = 8  # how many problems to attempt per step (parallel rollouts)

pbar = tqdm(range(training_steps), total=training_steps, desc="Training")

for step in pbar:
    # Collect trajectories from multiple parallel rollouts
    trajectories = [rollout(model) for _ in range(rollouts_per_step)]
    # Wait for all the asynchronous rollouts to finish
    trajectories = await asyncio.gather(*trajectories)

    # Log the mean reward of this batch for monitoring
    avg_reward = sum(t.reward for t in trajectories) / len(trajectories)
    wandb.log({"step": step, "average_reward": avg_reward})
    
    # 프로그레스바에 실시간 정보 표시
    pbar.set_postfix({
        'Step': step,
        'Avg_Reward': f'{avg_reward:.3f}',
        'Last_Rewards': [f'{t.reward:.1f}' for t in trajectories[-3:]]  # 마지막 3개만 표시
    })

Training: 100%|██████████| 50/50 [00:32<00:00,  1.55it/s, Step=49, Avg_Reward=-1.000, Last_Rewards=['-1.0', '-1.0', '-1.0']]


위 코드에서, 우린는 `rollout(model)` 을 동시에 8번 실행시켜 경험을 수집합니다. 그런 다음 (async context인 경우 `asyncio.gather`를 사용하여) 모든 항목을 await 하여 trajectories results의 list를 가져옵니다. 이런 trajectories는 agent의 interactons과 reward를 포함합니다. 이것들이 수집되면, ART의 backend는 내부적으로 training이 업데이트 될것입니다. (ART는 내부적으로 trajectories를 그룹화하고 GRPO 최적화 단계를 실행하여 모델의 policy를 개선합니다.)

우리는 또한 rollouts의 batch에 대해 reward 평균을 W&B에 로깅합니다. 이것은 training progress와 같은 것을 보기에 유용한 지표입니다.


또한 해당 rollouts의 batch에 대한 평균 reward를 W&B에 로깅합니다. 이는 관찰하기 유용한 지표입니다. 훈련이 진행됨에 따라 Agent가 성공적으로 학습하고 있다면 평균 reward가 1.0에 가까워질 것으로 예상할 수 있습니다.
우리의 곱셈 task 예시에서, 평균 reward 1.0은 에이전트가 해당 배치에서 모든 답을 올바르게 맞췄다는 것을 의미합니다.


## Step 5: Monitor training with Weave and W&B

training loop를 실행하면, 당신은 실시간으로 지표를 관측하기 위해 Weight & Biases project page로 이동 할 수 있습니다. 당신은 각 step별로 로깅된 training curve를 포함해 `average_reward` 지표를 볼 수 있습니다. 만일 당신이 ART에서 모델의 `project` 와 `name` 을 제대로 설정하였다면, ART는 아마 (loss 또는 policy entropy와 같은) 추가적인 지표를 자동으로 W&B에 로깅할 것입니다. 


이때 W&B Weave가 유용합니다. Weave를 사용하면 에이전트의 성능을 심층적으로 분석할 수 있는 interactive 대시보드를 만들 수 있습니다. 예를 들어, 단순히 reward curve를 보여주는 것뿐 아니라, 질문과 답변 샘플을 함께 표시하여 모델 출력이 어떻게 개선되는지를 확인할 수 있는 Weave 패널을 구축할 수 있습니다. 또, 곱셈 문제와 에이전트가 서로 다른 학습 단계에서 낸 답변을 표 형태로 나열하는 것도 가능하며, 이는 정성적 평가에 매우 유용합니다. Weave의 강점은 플롯, 텍스트, 심지어 모델 쿼리까지 한 공간에 결합할 수 있다는 점으로, AI 애플리케이션 분석을 위한 유연한 툴킷입니다.

또한 Weights & Biases에는 Models(모델 레지스트리 및 아티팩트)라는 기능도 있습니다. 학습이 끝나면, 예를 들어 ART가 학습한 LoRA 어댑터와 같은 파인튜닝된 모델을 저장하고 싶을 것입니다. W&B를 사용하면 이러한 결과물을 버저닝할 수 있습니다. 

In [9]:
# After training is done, save the LoRA weights 
final_lora_dir = f"./.art/{project_name}/models"  # hypothetical path where ART stored LoRA checkpoints
# Log the fine-tuned model as a W&B artifact for versioning
artifact = wandb.Artifact(name="multiplication-agent-lora", type="model")
artifact.add_dir(final_lora_dir)
wandb.log_artifact(artifact)


wandb: Adding directory to artifact (./.art/my-agentic-task/models)... 

Done. 0.3s


<Artifact multiplication-agent-lora>

아티팩트를 기록함으로써, 학습된 에이전트의 모델 가중치를 W&B에 영구적으로 보관할 수 있고, 이후에 이를 다시 불러오거나, 비교하거나, 심지어 배포할 수도 있습니다. W&B Models(모델 레지스트리)는 이러한 아티팩트와 그 계보(lineage)를 추적할 수 있어, 어떤 학습 실행(run)에서 어떤 모델이 생성되었는지를 명확히 알 수 있습니다.

## Step 6: Evaluate the trained agent

50번의 steps를 훈련시킨 이후라면, 이제 agent가 얼마나 잘 학습 되었는지 test 할 시간입니다.

`model.openai_client()`를 다시 사용하면 모델이 곱셈 문제에 답하고 정답인지 확인할 수 있습니다.

이상적으로는 이 모델이 처음보다 이 작업에서 훨씬 더 안정적이라는 것을 알게 될 것입니다.

훈련 후, 빠른 테스트를 위해, 아래 몇가지 샘플 쿼리를 시도하세요:

In [10]:
openai_client = model.openai_client()

In [11]:
test_questions = ["What is 7 * 8?", "What is 15 * 14?", "What is 3 * 19?"]


for q in test_questions:
    completion = await openai_client.chat.completions.create(
        model=model.name,
        messages=[{"role": "user", "content": q}]
    )
    answer = completion.choices[0].message.content.strip()
    print(f"Q: {q} -> A: {answer}")


Q: What is 7 * 8? -> A: 7 * 8 equals 56.
Q: What is 15 * 14? -> A: 15 * 14 equals 210.
Q: What is 3 * 19? -> A: 3 times 19 equals 57.


학습이 제대로 이루어졌다면, 위의 쿼리에 대한 답변이 정확히 나와야 합니다(예: `56, 210, 57`). 만약 일부가 틀리거나 정확도를 더 높이고 싶다면, 학습 스텝을 더 늘리거나 전략을 조정할 수 있습니다(예: 숫자의 범위를 넓히거나 보상 체계를 바꾸는 방법).

다른 활용 사례: 지금 살펴본 예시는 매우 단순하지만, OpenPipe ART는 다양한 에이전트 작업에 적용할 수 있습니다. 수학 문제 대신 “환경(environment)”을 `게임(예: 2048, 틱택토—ART로 이미 시연된 바 있음)`, `지식 검색 작업(에이전트가 도구를 사용해 정보를 찾고 요약)`, 혹은 `대화 워크플로우(에이전트가 긴 대화 속에서 지시를 따라야 하는 경우)`로 설정할 수도 있습니다. 과정은 대부분 동일합니다:

1. 에이전트가 처리해야 할 시나리오를 정의합니다.

2. ART 클라이언트를 통해 해당 시나리오에서 에이전트와 상호작용하고, 마지막에 보상을 부여하는 rollout 함수를 작성합니다.

3. 여러 번의 rollout을 실행해 ART가 모델의 동작을 최적화하도록 합니다.

보상 함수와 시나리오를 조정함으로써, 게임 플레이부터 대화 속 사실성이나 안전성 가이드라인 준수까지 다양한 기술을 에이전트에게 학습시킬 수 있습니다.

이 모든 과정에서 W&B는 실험 추적에 매우 유용합니다. 예를 들어, 다른 보상 함수를 사용했을 때 학습이 더 빨라졌는지 비교할 수 있고, 성능을 시각화하며, 최상의 모델을 관리할 수 있습니다. OpenPipe ART와 W&B의 통합 덕분에 하이퍼파라미터를 튜닝하거나 에이전트의 동작을 디버깅할 때 매우 풍부한 인사이트를 얻을 수 있습니다.

## Step 7: 모델 저장 및 허깅페이스에 업로드

In [ ]:
# 2. 허깅페이스에 업로드하기 위한 설정 (.env 파일 사용)
import os
from dotenv import load_dotenv

load_dotenv()

# .env 파일에서 API 키들을 로드
HF_TOKEN = os.getenv("HF_TOKEN")
HF_USERNAME = os.environ.get("HF_USERNAME")

repo_id = f"{HF_USERNAME}/{model.name}-ART"

print(repo_id)

In [15]:
## Step 7: 모델 저장 및 허깅페이스 업로드

# 1. 훈련된 모델을 로컬에 저장
import os
from huggingface_hub import HfApi, create_repo
import torch
from unsloth import FastLanguageModel

# ART에서 훈련된 LoRA 어댑터 저장
print("Saving trained model...")
lora_model_path = (
    f".art/{model.project}/models/{model.name}/checkpoints/{await model.get_step():04d}"
)

# ART 모델을 저장 (LoRA 어댑터 포함)
peft_model, peft_tokenizer = FastLanguageModel.from_pretrained(
    model_name=lora_model_path,
    max_seq_length=16384,
    dtype=torch.bfloat16,
    load_in_4bit=True,
)

# 허깅페이스 토큰 확인
if not HF_TOKEN:
    print("HUGGINGFACE_TOKEN이 .env 파일에 설정되지 않았습니다.")
    print("허깅페이스 업로드를 건너뜁니다.")

if HF_TOKEN and HF_USERNAME:
    try:
        # HuggingFace API 초기화
        peft_model.push_to_hub_merged(
            f"{HF_USERNAME}/{model.name}", peft_tokenizer, token=HF_TOKEN
        )
        
        print(f"✅ Model successfully uploaded to: https://huggingface.co/{repo_id}")
        
        # W&B에 허깅페이스 링크 로깅
        wandb.log({
            "huggingface_model_url": f"https://huggingface.co/{repo_id}",
            "final_model_saved": True
        })
        
    except Exception as e:
        print(f"❌ Error uploading to Hugging Face: {str(e)}")
        print("Please check your token and internet connection.")
        
else:
    print("⚠️  Hugging Face 토큰이 설정되지 않았습니다.")
    print("허깅페이스에 업로드하려면 다음 단계를 따르세요:")
    print("1. https://huggingface.co/settings/tokens 에서 토큰 생성")
    print("2. HF_TOKEN 변수에 실제 토큰 값 설정")
    print("3. HF_USERNAME을 본인의 허깅페이스 사용자명으로 변경")


print("\n🎉 모델 저장 및 업로드가 완료되었습니다!")
print("📊 W&B에서 훈련 진행상황을 확인하세요: https://wandb.ai")

# W&B 실행 종료
wandb.finish()

/tmp/ipykernel_450660/2341394760.py:7: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
Saving trained model...


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


==((====))==  Unsloth 2025.8.6: Fast Qwen2 patching. Transformers: 4.53.2. vLLM: 0.10.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 2. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.8.6 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.
Processing Files (1 / 1): 100%|██████████| 11.4MB / 11.4MB,  205kB/s  
New Data Upload: 100%|██████████|  329kB /  329kB,  205kB/s  


Found HuggingFace hub cache directory: /home/khw/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00,  1.09it/s]
Processing Files (1 / 1): 100%|██████████| 3.97GB / 3.97GB,  108MB/s  t/s]
New Data Upload: 100%|██████████|  275MB /  275MB,  693kB/s  
Processing Files (1 / 1): 100%|██████████| 2.20GB / 2.20GB,  133MB/s  , 163.40s/it]
New Data Upload: 100%|██████████| 7.44MB / 7.44MB,  744kB/s  
Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [04:06<00:00, 123.23s/it]


✅ Model successfully uploaded to: https://huggingface.co/HueyWoo/agent-001


## Step 8: 허깅페이스에 업로드한 모델 로드하고 테스트 하기

In [ ]:
# Unsloth로 허깅페이스 모델 로드 및 테스트

import torch
from dotenv import load_dotenv
import os
from unsloth import FastLanguageModel

load_dotenv()

# 환경변수에서 허깅페이스 사용자명 가져오기
HF_USERNAME = os.environ.get("HF_USERNAME")  # 기본값 설정
model_repo = f"{HF_USERNAME}/{model_name}"


# 결론

이 글에서는 OpenPipe ART를 기초적인 목적부터 실제 구현까지 깊이 있게 살펴보았습니다. 핵심 내용을 요약하면 다음과 같습니다. OpenPipe ART는 언어 모델 에이전트에 강화학습을 손쉽게 적용할 수 있게 해주는 강력한 프레임워크입니다. 이는 전통적인 강화학습 프레임워크의 여러 한계를 해결하는데, 다중 턴 에이전트 워크플로우를 지원하고, 클라이언트-서버 아키텍처를 통한 GPU 활용 극대화, OpenAI 스타일의 인터페이스를 통한 기존 코드베이스와의 손쉬운 통합이 그 예입니다. 
실무자에게 이는 곧, 성능이 부족하거나 실수를 반복하는 에이전트를 단순한 정적 파인튜닝이 아니라 경험을 통해 개선할 수 있음을 의미합니다. 그 결과로 신뢰성 향상(과거의 실수를 회피), 더 나은 과제 수행, 배포된 시스템에서의 지속 학습 가능성이라는 이점이 주어집니다.

우리는 ART 학습 루프를 설정하는 방법을 시연하고, Weights & Biases(W&B) 통합을 강조했습니다. W&B의 Weave와 Models를 활용하면 학습 진행 상황을 실시간으로 모니터링하고 학습된 모델을 관리할 수 있어, 실험 과정이 크게 향상됩니다. 단순한 곱셈 문제 학습 예시는 복잡한 작업을 다룰 때의 접근 방식을 단순화해 보여주는 것이었으며, 동일한 원리가 복잡한 의사결정이나 도구 사용과 같은 작업을 학습하는 데에도 적용됩니다. 즉, 작업을 정의하고, 보상으로 피드백을 제공하며, 나머지는 ART에 맡기면 됩니다.

앞으로를 내다보면, OpenPipe ART는 활발히 진화하고 있는 프로젝트입니다. 향후 발전 방향은 다음과 같을 수 있습니다:

강화된 보상 모델링: 현재는 보상을 직접 정의하거나 간단한 휴리스틱을 사용하지만, 앞으로는 학습된 보상 모델이나 인간 피드백(RLHF와 유사)을 직접 활용하는 고급 기법이 통합될 수 있습니다. 실제로 ART는 이미 RULER라는 기능을 도입해 LLM을 판정자로 활용, 경로를 점수화하고 수작업 보상 함수의 필요성을 크게 줄였습니다. 이러한 기능은 에이전트의 “성공” 기준 정의를 더 쉽게 만들 것입니다.

더 폭넓은 모델 지원과 확장성: 대규모 및 특수화 모델이 등장함에 따라 ART는 이를 지원하는 방향으로 확장될 것입니다. 프런트엔드와 백엔드의 분리 덕분에 분산 학습이나 대규모 모델의 하드웨어 샤딩까지 가능성이 열려 있으며, 새로운 하드웨어 최적화나 GRPO 알고리즘 효율성 개선도 기대할 수 있습니다.

프로덕션 통합: 현재 ART는 연구 및 프로토타이핑에 적합하지만, 장기적으로는 실제 서비스에 통합될 도구들이 등장할 수 있습니다. 예를 들어, 실제 사용자 상호작용 데이터를 기반으로 자동 학습하여(적절한 안전장치와 함께) 배포된 에이전트가 지속적으로 개선되는 방식입니다. 미래에는 이러한 지속 학습을 안전하고 간단하게 구현할 수 있는 기능이 추가될 가능성이 있습니다.

커뮤니티 기여와 생태계: 오픈소스 프로젝트인 만큼, ART의 발전은 사용자 커뮤니티의 기여에 크게 의존합니다. LangChain 같은 에이전트 오케스트레이션 라이브러리와의 통합, 커뮤니티 기반 시나리오 라이브러리, 더 많은 예제들이 생겨날 수 있습니다. 또한 W&B 생태계와의 결합을 통해 공개 대시보드, 리포트, 모범 사례 공유 등이 활발히 이루어질 것입니다.

OpenPipe ART는 대규모 언어 모델 기반 AI 에이전트에 강화학습을 실용적으로 적용하는 데 있어 중요한 진전을 보여줍니다. 개발자들은 정적인 모델 성능을 넘어, 시행착오를 통한 지속적인 행동 개선을 가능하게 할 수 있습니다. 자율 에이전트나 복잡한 챗봇을 만들고 있는데 기대만큼 강건하거나 정확하지 않다면, ART를 고려해보십시오. 최소한의 수정만으로 에이전트에 학습 루프를 내장할 수 있고, 점차 개선되는 모습을 확인할 수 있습니다. 또한 W&B 같은 도구로 학습 과정을 추적하면, 학습 과정을 완전히 투명하게 파악할 수 있습니다.

OpenPipe ART의 미래는 매우 밝습니다. 오픈소스 개발, 성장하는 사용자 커뮤니티, MLOps 도구와의 통합이 맞물려, 이 프레임워크는 앞으로 표준적인 에이전트 학습 접근 방식으로 자리 잡을 수 있습니다. 이는 자율적인 LLM 기반 에이전트가 할 수 있는 일의 범위를 계속해서 확장시킬 것입니다.

읽어주셔서 감사합니다. 즐거운 학습 되시길 바랍니다! 더 깊이 탐구하고 싶으시다면, 관련 예제를 확인하고 OpenPipe 커뮤니티에 참여해 보시길 권장합니다. 게임 플레이 에이전트든, 사용자 상호작용을 통해 학습하는 AI 어시스턴트든, OpenPipe ART와 W&B는 더 높은 성능을 달성하는 데 필요한 피드백 루프를 제공할 것입니다.